# Audit the new version of the persistence image class to ensure functionality in the scikit-TDA API, agreement with the previous version, and compare the speeds of the two implementations

## Additional native functionality:
* Images are no longer restricted to square regions of the persistence plane
* Non-isotrapic Gaussians are now supported
* One-parameter, nonlinear power functions now supported as weight functions

## Code updates:
* Summed-area table calculation of image

In [1]:
import numpy as np
import timeit
import matplotlib.pyplot as plt
from persim import PersImage
from persim import PersistenceImager

In [2]:
def random_pd(num_pairs=1000, birth_range=(0.0, 1.0), persistence_range=(0.0, 1.0), seed=None):
    """
    Generate a random persistence diagram over a specified range of the birth-death or birth-persistence plane
    """
    np.random.seed(seed)
    
    birth = np.random.uniform(low=birth_range[0], high=birth_range[1], size=(num_pairs,))
    persistence_range = np.random.uniform(low=persistence_range[0], high=persistence_range[1], size=(num_pairs,))
    death = persistence_range + birth
    pers_dgm = np.vstack((birth, death))
        
    return pers_dgm.transpose()

In [7]:
# Construct a default PersistenceImager object
persimager_new = PersistenceImager(pixel_size=.2)

# Construct an equivalent PersImage object
persimager_old = PersImage(pixels=(5, 5), spread=1.0, specs={
                "maxBD": 1,
                "minBD": 0
            }, weighting_type="linear", kernel_type="gaussian")

# Generate some random Persistence diagrams
num_pds = 50
pds = [random_pd(num_pairs=1000, birth_range=(0.0,1.0), persistence_range=(0.0,1.0), seed=0) for _ in range(num_pds)]

import timeit

start_time_new = timeit.default_timer()
pi_new = persimager_new.transform(pds, skew=True)
elapsed_new = timeit.default_timer() - start_time_new

start_time_old = timeit.default_timer()
pi_old = persimager_old.transform(pds)
elapsed_old = timeit.default_timer() - start_time_old

print('New code took %f sec.' %elapsed_new)
print('Old code took %f sec.' %elapsed_old)
print('%fX increase in performance.' %(elapsed_old / elapsed_new - 1))

PersImage(pixels=(5, 5), spread=1.0, specs={'maxBD': 1, 'minBD': 0}, kernel_type="gaussian", weighting_type="linear")
New code took 1.049817 sec.
Old code took 14.936982 sec.
13.228175X increase in performance.


In [7]:
pi_new.shape

(5, 5)

In [8]:
pi_old.shape

(5, 5)